In [1]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
import re
import string
import pandas as pd
import nltk

In [2]:
analyzer = SentimentIntensityAnalyzer()
def sentiment_analyzer_scores(text):
    score = analyzer.polarity_scores(text)
    print(score)

In [37]:
linedata = []
with open("../../tmp3.csv") as f:
    for line in f:
        linedata.append(line.split(','))

In [38]:
linedata = pd.DataFrame(linedata)

In [41]:
linedata['text'] = linedata[linedata.columns[6:]].fillna('').apply(lambda row: ''.join(row.values.astype(str)), axis = 1)

In [43]:
linedata.rename(columns = {0:'id', 1:'user', 2:'time', 4:'lat', 5:'lon'}, inplace = True)

In [44]:
df = linedata[['id','user','time','lat','lon','text']]

In [45]:
df.head()

,id,user,time,lat,lon,text
0,'262619667330396161','Andrewthemark','2012-10-28 18:19:42',u'coordinates': [40.67360978,-73.69137707]}','@yeahimjailed @marcushaight yeah but kristin ...
1,'262619681305792512','cabrera','2012-10-28 18:19:45',u'coordinates': [40.6879886,-73.9546139]}','@zarabeatriz @dupre se viene Sandy.'\n
2,'262619684543811584','m0ff','2012-10-28 18:19:46',u'coordinates': [45.27374765,-75.86502686]}','I wonder how bad my migraine is gonna be when...
3,'262619693376999424','Melbroadcasts','2012-10-28 18:19:48',u'coordinates': [40.77172216,-73.91315814]}','Come on atleast save some of the wolves! http...
4,'262619694199107584','LeEmLeGiT','2012-10-28 18:19:48',u'coordinates': [41.11522062,-75.04597037]}',"'""@jellii19: Sandy #YouScared... Roll through ..."


In [46]:
df = df.dropna()

In [47]:
df['lat'] = df['lat'].str.split('[').apply(lambda x: x[1])

In [48]:
df['lon'] = df['lon'].str.split(']').apply(lambda x: x[0])

In [49]:
def preprocess(text):
    if 'via' in text:
        return re.match('(.*?)via.*?\s?(.*?)', re.sub(r"http\S+", "", re.sub(r"[^a-zA-Z0-9]+", ' ', re.sub(r'\d+', '', text.lower()).translate(str.maketrans('', '', string.punctuation)).strip()))).group(1)
    else:
        return re.sub(r"http\S+", "", re.sub(r"[^a-zA-Z0-9]+", ' ', re.sub(r'\d+', '', text.lower()).translate(str.maketrans('', '', string.punctuation)).strip()))

In [25]:
from pycorenlp import StanfordCoreNLP 

In [26]:
nlp = StanfordCoreNLP('http://localhost:9000')

In [31]:
# def get_sentiment(text):
#     res = nlp.annotate(text,
#                        properties={'annotators':'sentiment',
#             'outputFormat': 'json',
#             'timeout': 1000,
# })
#     print(text)
#     print('Sentiment:', res['sentences'][0]['sentiment'])
#     print('Sentiment score:', res['sentences'][0]['sentimentValue'])
#     print('Sentiment distribution (0-v. negative, 5-v.positive:', res['sentences'][0]['sentimentDistribution'])

In [44]:
def get_sentiment(text):
    res = nlp.annotate(text,
                       properties={'annotators':'sentiment',
            'outputFormat': 'json',
            'timeout': 10000,
})
    return(res['sentences'][0]['sentiment'],res['sentences'][0]['sentimentValue'])

In [36]:
df['process_text'] = df['text'].apply(preprocess)

In [46]:
df['sentiment'] = df['process_text'].apply(get_sentiment)

In [48]:
df.head()

,id,user,time,lat,lon,text,process_text,sentiment
0,'262619667330396161','Andrewthemark','2012-10-28 18:19:42',40.67360978,-73.69137707,'@yeahimjailed @marcushaight yeah but kristin ...,yeahimjailed marcushaight yeah but kristin the...,"(Negative, 1)"
1,'262619681305792512','cabrera','2012-10-28 18:19:45',40.6879886,-73.9546139,'@zarabeatriz @dupre se viene Sandy.'\n,zarabeatriz dupre se viene sandy,"(Negative, 1)"
2,'262619684543811584','m0ff','2012-10-28 18:19:46',45.27374765,-75.86502686,'I wonder how bad my migraine is gonna be when...,i wonder how bad my migraine is gonna be when ...,"(Negative, 1)"
3,'262619693376999424','Melbroadcasts','2012-10-28 18:19:48',40.77172216,-73.91315814,'Come on atleast save some of the wolves! http...,come on atleast save some of the wolves hopey...,"(Negative, 1)"
4,'262619694199107584','LeEmLeGiT','2012-10-28 18:19:48',41.11522062,-75.04597037,"'""@jellii19: Sandy #YouScared... Roll through ...",jellii sandy youscared roll through the dirty ...,"(Negative, 1)"
